In [58]:
import pandas as pd
import numpy as np

# Netflix Data Cleaning and Preprocessing

This notebook performs data cleaning and preprocessing on the Netflix titles dataset (`data/netflix_titles.csv`). The goal is to handle missing values, standardize formats, remove duplicates, and prepare the data for exploratory data analysis.

**Steps Covered:**
- Data Loading and Initial Inspection
- Missing Value Handling
- Date Conversion and Temporal Feature Extraction
- Duration Standardization
- Data Validation and Saving

## 1. Data Loading and Initial Inspection

Load the dataset and perform initial checks for shape, data types, and missing values.

In [59]:
df = pd.read_csv('data/netflix_titles.csv', index_col='show_id')
print(df.shape)
df.head()

(8807, 11)


,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [60]:
print("\nMissing values rates before inputation:")
for column in df.columns:
    null_counts = df[column].isnull().sum()
    null_rate =  null_counts / len(df) * 100 
    if null_rate > 0 :
        print(f"{column} null rate: {round(null_rate, 2)}%, null counts: {null_counts}")


Missing values rates before inputation:
director null rate: 29.91%, null counts: 2634
cast null rate: 9.37%, null counts: 825
country null rate: 9.44%, null counts: 831
date_added null rate: 0.11%, null counts: 10
rating null rate: 0.05%, null counts: 4
duration null rate: 0.03%, null counts: 3


## 2. Missing Value Handling

Impute missing values using appropriate strategies (e.g., mode for categorical, median for numerical) and remove duplicates.

In [61]:
mode_country = df['country'].mode()
if not mode_country.empty:
    df['country'] = df['country'].fillna(mode_country[0])
else:
    df['country'] = df['country'].fillna('Unknown')

# Impute date_added with 'January 1, ' + release_year
df['date_added'] = df['date_added'].fillna('January 1, ' + df['release_year'].astype(str))

df['cast'] = df['cast'].replace([np.nan, None, ''], 'No Data')
df['director'] = df['director'].replace([np.nan, None, ''], 'No Data')
df['rating'] = df['rating'].replace([np.nan, None, ''], 'No Data')

df = df.dropna()
df = df.drop_duplicates(subset=['title'], keep='first')

# Verify no missing values remain
print("Missing values after imputation:")
print(df.isnull().sum())

Missing values after imputation:
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64


In [62]:
# Check rows where date_added is NaT after conversion
nat_rows = df[df['date_added'].isna()]
print("Number of rows with NaT in date_added:", len(nat_rows))
if len(nat_rows) > 0:
    print("Sample rows with NaT:")
    print(nat_rows[['title', 'date_added', 'release_year']].head(10))
    # To handle separately, e.g., set to release_year as date
    # df.loc[df['date_added'].isna(), 'date_added'] = pd.to_datetime(df['release_year'].astype(str) + '-01-01')
else:
    print("No NaT values found.")

Number of rows with NaT in date_added: 0
No NaT values found.


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8804 entries, s1 to s8807
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8804 non-null   object
 1   title         8804 non-null   object
 2   director      8804 non-null   object
 3   cast          8804 non-null   object
 4   country       8804 non-null   object
 5   date_added    8804 non-null   object
 6   release_year  8804 non-null   int64 
 7   rating        8804 non-null   object
 8   duration      8804 non-null   object
 9   listed_in     8804 non-null   object
 10  description   8804 non-null   object
dtypes: int64(1), object(10)
memory usage: 1.1+ MB


## 3. Date Conversion and Temporal Feature Extraction

Convert `date_added` to datetime format and extract temporal components (month, year) for analysis.

In [64]:
# Convert date_added to datetime using dateutil for better parsing
from dateutil import parser

def safe_parse(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        return parser.parse(date_str)
    except:
        return pd.NaT

df['date_added'] = df['date_added'].apply(safe_parse)
df['date_added'] = pd.to_datetime(df['date_added'])  # Ensure it's datetime64

# Extract temporal components from date_added for analysis
df['month_added'] = df['date_added'].dt.month
df['month_name_added'] = df['date_added'].dt.month_name()
df['year_added'] = df['date_added'].dt.year

print("Temporal columns added. Sample:")
print(df[['date_added', 'month_added', 'month_name_added', 'year_added']].head())

print("Data types after conversion:")
print(df.dtypes)

Temporal columns added. Sample:
        date_added  month_added month_name_added  year_added
show_id                                                     
s1      2021-09-25            9        September        2021
s2      2021-09-24            9        September        2021
s3      2021-09-24            9        September        2021
s4      2021-09-24            9        September        2021
s5      2021-09-24            9        September        2021
Data types after conversion:
type                        object
title                       object
director                    object
cast                        object
country                     object
date_added          datetime64[ns]
release_year                 int64
rating                      object
duration                    object
listed_in                   object
description                 object
month_added                  int32
month_name_added            object
year_added                   int32
dtype: object


In [65]:
# Make sure no NaT in date_added after converted to datetime64
print(df.isnull().sum())
df[df['date_added'].isnull()][['title', 'release_year', 'date_added', 'month_added', 'month_name_added', 'year_added']].head()

type                0
title               0
director            0
cast                0
country             0
date_added          0
release_year        0
rating              0
duration            0
listed_in           0
description         0
month_added         0
month_name_added    0
year_added          0
dtype: int64


,title,release_year,date_added,month_added,month_name_added,year_added
show_id,,,,,,


## 4. Duration Standardization

Extract numeric values from `duration` for movies (minutes) and TV shows (seasons), handling them separately.

In [66]:
# Extract numeric duration separately for movies and TV shows
# For movies: duration in minutes
df['duration_minutes'] = df.apply(lambda x: int(x['duration'].split()[0]) if pd.notna(x['duration']) and 'min' in x['duration'] else np.nan, axis=1).astype('Int64')

# For TV shows: duration in seasons
df['duration_seasons'] = df.apply(lambda x: int(x['duration'].split()[0]) if pd.notna(x['duration']) and 'Season' in x['duration'] else np.nan, axis=1).astype('Int64')

# Fill NaN with median for each
median_minutes = df['duration_minutes'].median()
df['duration_minutes'] = df['duration_minutes'].fillna(median_minutes)

median_seasons = df['duration_seasons'].median()
df['duration_seasons'] = df['duration_seasons'].fillna(median_seasons)

print("Duration columns standardized. Sample:")
print(df[['type', 'duration', 'duration_minutes', 'duration_seasons']].head(10))

Duration columns standardized. Sample:
            type   duration  duration_minutes  duration_seasons
show_id                                                        
s1         Movie     90 min                90                 1
s2       TV Show  2 Seasons                98                 2
s3       TV Show   1 Season                98                 1
s4       TV Show   1 Season                98                 1
s5       TV Show  2 Seasons                98                 2
s6       TV Show   1 Season                98                 1
s7         Movie     91 min                91                 1
s8         Movie    125 min               125                 1
s9       TV Show  9 Seasons                98                 9
s10        Movie    104 min               104                 1


## 5. Data Validation and Saving

Validate the cleaned dataset and save it for use in subsequent notebooks.

In [67]:
# Save cleaned dataset
df.to_csv('data/netflix_titles_cleaned.csv', index=False)
print("Cleaned dataset saved as 'data/netflix_titles_cleaned.csv'")

Cleaned dataset saved as 'data/netflix_titles_cleaned.csv'


## Summary

The dataset has been cleaned and preprocessed:
- Missing values imputed or filled.
- Dates converted to datetime with temporal features extracted.
- Duration standardized into separate numeric columns.
- Duplicates removed.

The cleaned data is saved as `data/netflix_titles_cleaned.csv` and ready for exploratory data analysis in the next notebook.

**Next Steps:**
- Proceed to `exploratory_data_analysis_and_visualization.ipynb` for insights and visualizations.
- Use the new columns (e.g., `duration_minutes`, `month_added`) for advanced analysis.